In [1]:
import pandas as pd 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate


In [2]:
data= pd.read_csv("D:\PFE\Fact_Table_Rev.csv", low_memory=False)

In [3]:
data

,montant_HT,montant_ttc,date_saisie,date_facture,Fk_prestation,Fk_famille_prestation,Fk_nationnalite,Fk_Societe,Fk_typesociete
0,4.0,3.571429,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
1,6.0,5.357143,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
2,8.0,7.142857,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
3,10.0,8.928571,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
4,8.5,7.589286,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
...,...,...,...,...,...,...,...,...,...
937562,10.5,9.375000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937563,7.0,6.250000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937564,3.5,3.125000,2013-09-24,2013-09-24,BOIR,CAVE,TN,STANDART,SOC
937565,196.0,175.000000,2013-09-24,2013-09-24,REP,RESTO,TN,STANDART,SOC


In [3]:
df = pd.DataFrame(data, columns=["montant_HT", "date_saisie", "Fk_famille_prestation","Fk_typesociete"])

In [4]:
df['date_saisie'] = pd.to_datetime(df['date_saisie'])

In [6]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete
0,4.0,2006-09-18,BAR,SOC
1,6.0,2006-09-18,BAR,SOC
2,8.0,2006-09-18,BAR,SOC
3,10.0,2006-09-18,BAR,SOC
4,8.5,2006-09-18,BAR,SOC
...,...,...,...,...
937562,10.5,2013-09-24,BAR,LAB
937563,7.0,2013-09-24,BAR,LAB
937564,3.5,2013-09-24,CAVE,SOC
937565,196.0,2013-09-24,RESTO,SOC


In [7]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
1              6.0  2006-09-18                   BAR            SOC
2              8.0  2006-09-18                   BAR            SOC
3             10.0  2006-09-18                   BAR            SOC
4              8.5  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
937562        10.5  2013-09-24                   BAR            LAB
937563         7.0  2013-09-24                   BAR            LAB
937564         3.5  2013-09-24                  CAVE            SOC
937565       196.0  2013-09-24                 RESTO            SOC
937566         3.5  2013-09-24                  CAVE            PET

[937567 rows x 4 columns]


In [5]:
df= df.drop(index=df.loc[(df['date_saisie'] < '2006-01-01') | (df['date_saisie'] > '2019-12-31')].index)

In [6]:
df= df.sort_values(by='date_saisie', ascending=True)

In [129]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
313            0.0  2006-09-18                   BAR            SOC
314            0.0  2006-09-18                   BAR            SOC
315            0.0  2006-09-18                   BAR            SOC
316            0.0  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
765368       120.0  2019-12-31                TORRET            SOC
765367       147.0  2019-12-31                TORRET            SOC
765366        16.0  2019-12-31                TORRET            SOC
765364        44.0  2019-12-31                TORRET            SOC
765337        12.0  2019-12-31                TORRET            SOC

[753932 rows x 4 columns]


In [7]:
label_encoder = LabelEncoder()
df['Fk_prestation_code'] = label_encoder.fit_transform(df['Fk_famille_prestation'])
label_encoder = LabelEncoder()
df['Fk_typesociete_code'] = label_encoder.fit_transform(df['Fk_typesociete'])
df.head()

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5


In [10]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5
...,...,...,...,...,...,...
765368,120.0,2019-12-31,TORRET,SOC,19,5
765367,147.0,2019-12-31,TORRET,SOC,19,5
765366,16.0,2019-12-31,TORRET,SOC,19,5
765364,44.0,2019-12-31,TORRET,SOC,19,5


In [9]:
df1 = df[['Fk_famille_prestation', 'Fk_prestation_code']].drop_duplicates()
print(df1)

       Fk_famille_prestation  Fk_prestation_code
0                        BAR                   2
324                     CAVE                   3
910                    RESTO                  15
339                   SEJOUR                  17
5044                     PTD                  14
224                       MB                   9
1725                     LIN                   6
6484                     TEL                  18
2416                    DEBO                   4
33558                    P.S                  10
12757                  LOCAT                   7
96329                    DIV                   5
9208                    PISC                  12
18717                   PEIG                  11
94129                 BALNEO                   0
140920                S.COIF                  16
136211                    PR                  13
208095                   LOY                   8
416562                BANQUE                   1
456896              

Linear Regression Model

In [461]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Supposons que vous avez vos données dans un DataFrame pandas appelé 'data'

# Séparation des features et de la target
X = df[['Fk_prestation_code', 'Fk_typesociete_code']]
y = df['montant_HT']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)


In [374]:
model = LinearRegression()

In [375]:
model.fit(X_train, y_train)

LinearRegression()

In [376]:
y_pred = model.predict(X_test)

In [377]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 90084.10141799343


In [402]:
X_test

,Fk_prestation_code,Fk_typesociete_code
727644,16,5
727643,16,5
727645,16,5
727940,16,5
727647,16,5
...,...,...
765325,12,4
765322,12,5
765492,12,5
765498,12,4



Linear Regression 2


In [8]:
X_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)][['Fk_prestation_code']]
X_test = df[df['date_saisie'].dt.year == 2019][['Fk_prestation_code']]
y_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)]['montant_HT']
y_test = df[df['date_saisie'].dt.year == 2019]['montant_HT']

In [472]:
model = LinearRegression()

In [473]:
model.fit(X_train, y_train)

LinearRegression()

In [474]:
y_pred = model.predict(X_test)

In [475]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 138449.61117165224


In [476]:
y_pred.sum()

4831987.399383133

In [477]:
result_df = pd.DataFrame({'y_pred': y_pred, 'Fk_prestation_code': X_test['Fk_prestation_code']})
sum_by_prestation_code = result_df.groupby('Fk_prestation_code')['y_pred'].sum()


In [478]:
mapping_df = df[['Fk_prestation_code', 'Fk_famille_prestation']].drop_duplicates()

# Merge total_montant_HT_by_code with mapping_df on Fk_prestation_code
total_montant_HT_with_famille = sum_by_prestation_code.reset_index().merge(mapping_df, on='Fk_prestation_code')

# Display the result
print(total_montant_HT_with_famille)

    Fk_prestation_code        y_pred Fk_famille_prestation
0                    1  1.053749e+02                BANQUE
1                    2  1.062049e+05                   BAR
2                    3  9.759502e+04                  CAVE
3                    4  3.360634e+02                  DEBO
4                    5  5.374443e+01                   DIV
5                    6  7.468102e+04                   LIN
6                    8  5.337238e+02                   LOY
7                    9  1.970513e+05                    MB
8                   12  1.712340e+04                  PISC
9                   14  7.584563e+05                   PTD
10                  15  1.088685e+06                 RESTO
11                  17  1.050454e+06                SEJOUR
12                  18  1.124439e+04                   TEL
13                  19  1.429464e+06                TORRET


In [133]:
total_montant_HT_with_famille.to_csv('total_montant_HT_with_famille.csv', index=False)

Random Forest Regressor Model


In [15]:
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    bootstrap=True,
    random_state=42,
    n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, min_samples_leaf=5, min_samples_split=10,
                      n_jobs=-1, random_state=42)

In [16]:
y_pred = rf.predict(X_test)

In [17]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 135000.09673838608


In [18]:
y_pred.sum()

4466807.579831485

In [19]:
result_df = pd.DataFrame({'y_pred': y_pred, 'Fk_prestation_code': X_test['Fk_prestation_code']})
sum_by_prestation_code = result_df.groupby('Fk_prestation_code')['y_pred'].sum()
mapping_df = df[['Fk_prestation_code', 'Fk_famille_prestation']].drop_duplicates()

# Merge total_montant_HT_by_code with mapping_df on Fk_prestation_code
Predfamilleprestation= sum_by_prestation_code.reset_index().merge(mapping_df, on='Fk_prestation_code')

# Display the result
print(Predfamilleprestation)

    Fk_prestation_code        y_pred Fk_famille_prestation
0                    1  7.140806e+04                BANQUE
1                    2  1.336201e+05                   BAR
2                    3  1.623139e+05                  CAVE
3                    4  1.855180e+03                  DEBO
4                    5  3.111798e+02                   DIV
5                    6  2.849859e+04                   LIN
6                    8  2.375100e+04                   LOY
7                    9  1.493995e+04                    MB
8                   12  6.763834e+03                  PISC
9                   14  2.041393e+05                   PTD
10                  15  1.860729e+06                 RESTO
11                  17  1.299732e+06                SEJOUR
12                  18  5.276139e+02                   TEL
13                  19  6.582181e+05                TORRET


In [42]:
Predfamilleprestation.to_csv('Predfamilleprestation.csv', index=False)

SARIMAX

In [271]:
selected_columns = ['date_saisie', 'montant_HT','Fk_prestation_code']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['date_saisie','year','month', 'day','montant_HT','Fk_prestation_code']
df1 = df1.loc[:, selected_columns]

In [97]:
df1

,date_saisie,year,month,day,montant_HT,Fk_prestation_code
0,2006-09-18,2006,9,18,4.0,2
313,2006-09-18,2006,9,18,0.0,2
314,2006-09-18,2006,9,18,0.0,2
315,2006-09-18,2006,9,18,0.0,2
316,2006-09-18,2006,9,18,0.0,2
...,...,...,...,...,...,...
765368,2019-12-31,2019,12,31,120.0,19
765367,2019-12-31,2019,12,31,147.0,19
765366,2019-12-31,2019,12,31,16.0,19
765364,2019-12-31,2019,12,31,44.0,19


In [272]:
df1= df1.groupby([ 'date_saisie','Fk_prestation_code'])['montant_HT'].sum().reset_index()

In [273]:
df1.set_index('date_saisie', inplace=True)

In [141]:
df1

,Fk_prestation_code,montant_HT
date_saisie,,
2006-09-18,2,111.0
2006-09-18,3,0.0
2006-09-19,2,66.0
2006-09-19,3,26.0
2006-09-19,15,171.0
...,...,...
2019-12-31,12,15.0
2019-12-31,14,548.0
2019-12-31,15,9630.5


In [274]:
# Define the codes to remove
codes_to_remove = [10, 0,19,11,16,7,13,8,]  # Add more codes here if needed

# Remove rows where Fk_prestation_code matches any of the specified codes
df1 =df1[~df1['Fk_prestation_code'].isin(codes_to_remove)]

In [225]:
index_type = type(df1.index)

# Check if it's a DateTimeIndex
is_datetime_index = isinstance(df1.index, pd.DatetimeIndex)

# Print the result
print("Index type:", index_type)
print("Is DateTimeIndex:", is_datetime_index)

Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Is DateTimeIndex: True


In [275]:
train_data = df1.loc[:'2018']
test_data = df1.loc['2019':]

In [86]:
train_data

,Fk_prestation_code,montant_HT
date_saisie,,
2006-09-18,2,111.000
2006-09-18,3,0.000
2006-09-19,2,66.000
2006-09-19,3,26.000
2006-09-19,15,171.000
...,...,...
2018-12-31,14,198.037
2018-12-31,15,5561.916
2018-12-31,17,1023.363


In [276]:
all_prestation1 = test_data.Fk_prestation_code.unique()
all_prestation1

array([ 2,  3,  6,  9, 14, 15, 17, 18, 12,  1,  5,  4])

In [216]:
all_prestation2 = train_data.Fk_prestation_code.unique()
all_prestation2

array([1])

In [95]:
test_data

,date,Fk_prestation_code,montant_HT
0,2019-01-01,2,320.673
1,2019-01-01,3,10.000
2,2019-01-01,6,11.000
3,2019-01-01,9,4.673
4,2019-01-01,14,185.907
...,...,...,...
3131,2019-12-31,12,15.000
3132,2019-12-31,14,548.000
3133,2019-12-31,15,9630.500
3134,2019-12-31,17,2790.400


In [277]:
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")


results = list()
all_prestation = df1.Fk_prestation_code.unique()
for i in tqdm(range(len(all_prestation))):
    prestation = all_prestation[i]
    sub_df = df1[df1.Fk_prestation_code == prestation]
    train_data = sub_df.loc['2015':'2018']
    test_data = sub_df.loc['2019':]
    model = SARIMAX(train_data['montant_HT'],  order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    result = model.fit()
    start = len(train_data)
    end = len(train_data) + len(test_data) -1
    predictions = list(result.predict(start, end))
    prestation_result = pd.DataFrame()
    prestation_result["montant_predit"] = predictions
    prestation_result["date"] = list(test_data.index)
    prestation_result["Fk_prestation_code"] = prestation
    results.append(prestation_result)

100%|██████████| 12/12 [00:29<00:00,  2.46s/it]


In [278]:
final_pred = pd.concat(results)

In [266]:
final_pred

,montant_predit,date,Fk_prestation_code
0,4342.769557,2019-01-01,19
1,4351.362238,2019-01-02,19
2,4267.051406,2019-01-03,19
3,4857.487220,2019-01-04,19
4,4445.681408,2019-01-05,19
...,...,...,...
322,5173.763396,2019-12-27,19
323,5253.765248,2019-12-28,19
324,5221.856448,2019-12-29,19
325,5222.652346,2019-12-30,19


In [279]:
test_data = df1.loc['2019':].reset_index()
test_data.rename(columns= {"date_saisie":'date'},inplace=True)

In [56]:
final_pred.dtypes

montant_predit        float64
date                    int64
Fk_prestation_code      int32
dtype: object

In [57]:
test_data.dtypes

index                          int64
date                  datetime64[ns]
Fk_prestation_code             int32
montant_HT                   float64
dtype: object

In [280]:
merged_data=pd.merge(left = test_data, right = final_pred, how = 'left', on = ['date','Fk_prestation_code'])

In [281]:
merged_data['montant_predit'].sum()

4488874.824142836

In [283]:
sum_by_code = merged_data.groupby('Fk_prestation_code').agg({
    'montant_predit': 'sum',
    'montant_HT':'sum'
}).reset_index()
print(sum_by_code)

    Fk_prestation_code  montant_predit    montant_HT
0                    1    9.588220e+04  6.193900e+04
1                    2    1.732459e+05  1.892159e+05
2                    3    2.243414e+05  2.209189e+05
3                    4    1.015000e+04  4.812000e+03
4                    5   -1.563560e+03  5.190000e+02
5                    6    1.008829e+04  3.880193e+04
6                    9    1.130705e+04  2.191949e+04
7                   12    8.851961e+03  1.349750e+04
8                   14    2.719536e+05  2.793753e+05
9                   15    2.313909e+06  2.109300e+06
10                  17    1.369166e+06  1.307727e+06
11                  18    1.542342e+03  8.759200e+02


In [197]:
sum_by_code.to_csv('Predsarimax.csv', index=False)

In [13]:
model = SARIMAX(train_data['montant_HT'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
result = model.fit() 

d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [17]:
start = len(train_data)
end = len(train_data) + len(test_data) -1
predictions = result.predict(start, end)

d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [18]:
predictions.sum()

5079910.619187778

In [26]:
# Assuming 'predictions' contains the SARIMAX predictions and 'test_data' is your DataFrame
# You need to adjust this based on your actual DataFrame structure

# Create a DataFrame to store predictions with the corresponding Fk_prestation_code
predictions_df = test_data.copy()
predictions_df['Predictions'] = predictions

# Group predictions by Fk_prestation_code and sum the values
sum_predictions = predictions_df.groupby('Fk_prestation_code')['Predictions'].sum().reset_index()

# Print the sum of predictions for each Fk_prestation_code
print(sum_predictions)



    Fk_prestation_code  Predictions
0                    1          0.0
1                    2          0.0
2                    3          0.0
3                    4          0.0
4                    5          0.0
5                    6          0.0
6                    8          0.0
7                    9          0.0
8                   12          0.0
9                   14          0.0
10                  15          0.0
11                  17          0.0
12                  18          0.0
13                  19          0.0


LSTM


In [155]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [137]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5
...,...,...,...,...,...,...
765368,120.0,2019-12-31,TORRET,SOC,19,5
765367,147.0,2019-12-31,TORRET,SOC,19,5
765366,16.0,2019-12-31,TORRET,SOC,19,5
765364,44.0,2019-12-31,TORRET,SOC,19,5


In [253]:
selected_columns = ['date_saisie', 'montant_HT','Fk_prestation_code']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['year','month', 'day','montant_HT','Fk_prestation_code']
df1 = df1.loc[:, selected_columns]
df1

,year,month,day,montant_HT,Fk_prestation_code
0,2006,9,18,4.0,2
313,2006,9,18,0.0,2
314,2006,9,18,0.0,2
315,2006,9,18,0.0,2
316,2006,9,18,0.0,2
...,...,...,...,...,...
765368,2019,12,31,120.0,19
765367,2019,12,31,147.0,19
765366,2019,12,31,16.0,19
765364,2019,12,31,44.0,19


In [254]:
train_data = df1[(df1['year'] <= 2018)]
test_data = df1[df1['year'] == 2019]

In [255]:
train_data

,year,month,day,montant_HT,Fk_prestation_code
0,2006,9,18,4.0,2
313,2006,9,18,0.0,2
314,2006,9,18,0.0,2
315,2006,9,18,0.0,2
316,2006,9,18,0.0,2
...,...,...,...,...,...
729061,2018,12,31,20.0,9
729062,2018,12,31,13.0,6
729063,2018,12,31,10.0,3
727937,2018,12,31,13.0,19


In [256]:
train_data = train_data.groupby([ 'year','month','day' ,'Fk_prestation_code'])['montant_HT'].sum().reset_index()
test_data  = test_data .groupby([ 'year','month','day', 'Fk_prestation_code'])['montant_HT'].sum().reset_index()

In [257]:
train_data

,year,month,day,Fk_prestation_code,montant_HT
0,2006,9,18,2,111.000
1,2006,9,18,3,0.000
2,2006,9,19,2,66.000
3,2006,9,19,3,26.000
4,2006,9,19,15,171.000
...,...,...,...,...,...
37303,2018,12,31,14,198.037
37304,2018,12,31,15,5561.916
37305,2018,12,31,17,1023.363
37306,2018,12,31,18,8.080


In [224]:
print(test_data)

      year  month  Fk_prestation_code  montant_HT
0     2019      1                   2     320.673
1     2019      1                   3      10.000
2     2019      1                   6      11.000
3     2019      1                   9       4.673
4     2019      1                  14     185.907
...    ...    ...                 ...         ...
3143  2019     12                  12      15.000
3144  2019     12                  14     548.000
3145  2019     12                  15    9630.500
3146  2019     12                  17    2790.400
3147  2019     12                  19    2898.000

[3148 rows x 4 columns]


new LSTM Method

In [100]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Input, concatenate, Embedding, Flatten
from sklearn.preprocessing import MinMaxScaler

In [162]:
selected_columns = ['date_saisie', 'montant_HT','Fk_prestation_code']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['year','month','day','montant_HT','Fk_prestation_code']
df1 = df1.loc[:, selected_columns]
df1

,year,month,day,montant_HT,Fk_prestation_code
0,2006,9,18,4.0,2
313,2006,9,18,0.0,2
314,2006,9,18,0.0,2
315,2006,9,18,0.0,2
316,2006,9,18,0.0,2
...,...,...,...,...,...
765368,2019,12,31,120.0,19
765367,2019,12,31,147.0,19
765366,2019,12,31,16.0,19
765364,2019,12,31,44.0,19


In [163]:
df1 = df1.groupby([ 'year','month','day','Fk_prestation_code'])['montant_HT'].sum().reset_index()

In [164]:
df1

,year,month,day,Fk_prestation_code,montant_HT
0,2006,9,18,2,111.0
1,2006,9,18,3,0.0
2,2006,9,19,2,66.0
3,2006,9,19,3,26.0
4,2006,9,19,15,171.0
...,...,...,...,...,...
40451,2019,12,31,12,15.0
40452,2019,12,31,14,548.0
40453,2019,12,31,15,9630.5
40454,2019,12,31,17,2790.4


In [165]:
train_data = df1[(df1['year'] <= 2018)]
test_data = df1[df1['year'] == 2019]

In [166]:
X_train = train_data.drop(['year','montant_HT','day','month'], axis=1)
y_train=train_data['montant_HT']
X_train_padded = train_data.drop(['montant_HT'], axis=1)
y_train_padded = train_data['montant_HT']

X_test = test_data.drop(['year','montant_HT','month','day'], axis=1)
X_test_padded = test_data.drop(['montant_HT'], axis=1)
y_test_padded = test_data['montant_HT']
y_test=test_data['montant_HT']


In [167]:
input_type = Input(shape=(1,), name='input_type')
embedding_type = Dense(64, activation='relu')(input_type)

In [168]:
seq_length=12
input_time = Input(shape=(seq_length, X_train_padded.shape[2]), name='input_time')
lstm_time = LSTM(units=64)(input_time)

IndexError: tuple index out of range

In [112]:
concatenated = concatenate([embedding_type, lstm_time])

In [113]:
output = Dense(1, name='output')(concatenated)

In [114]:
model = Model(inputs=[input_type, input_time], outputs=output)

In [115]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [118]:
model.fit({'input_type': X_train['Fk_prestation_code'], 'input_time': X_train_padded}, y_train_padded, epochs=10, batch_size=32)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
predictions = model.predict({'input_type': X_test['Type_prestation'], 'input_time': X_test_padded})

In [128]:
predictions.sum()

3388791.5